In [1]:
import os
import sys
import json
import torch
import torchvision
from tqdm import tqdm_notebook as tqdm
from transformers import BertModel, BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I0224 06:15:29.878286 140557905581888 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
from fgc_support_retri.ser_extractor import *
from fgc_support_retri.utils import read_fgc, read_hotpot, json_load, json_write
from fgc_support_retri import config
from evaluation.eval import eval_sp_fgc, eval_fgc_atype

In [4]:
extractor = Syn_extractor("syn-all")

I0224 06:15:31.310876 140557905581888 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0224 06:15:32.201987 140557905581888 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0224 06:15:32.204940 140557905581888 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

In [5]:
data = json_load(config.FGC_TEST)

In [36]:
all_items = []
for d in tqdm(data):
    for q in d['QUESTIONS']:
        item = {'QID': q['QID'], 
               'SENTS': d['SENTS'],
               'QTEXT': q['QTEXT_CN'], 
               'SUP_EVIDENCE': q['SHINT'], 
               'ATYPE': q['ATYPE']}
        all_items.append(item)


100%|██████████| 25/25 [00:00<00:00, 15989.27it/s]

In [8]:
all_sp_predictions = []
all_atype_predictions = []
for d in tqdm(data):
    item = extractor.get_item(d)
    items= [i for i in item]
    predictions = extractor.predict(items)
    all_sp_predictions += predictions

100%|██████████| 25/25 [03:05<00:00, 11.75s/it]


In [21]:
qids = []
questions = []
t_sp = []
p_sp = []

for document in data:
    for question in document['QUESTIONS']:
        qids.append(question['QID'])
        questions.append(question['QTEXT_CN'])
        t_sp_text = []
        p_sp_text = []
        for idx in question['SHINT']:
            t_sp_text.append(document['SENTS'][idx]['text'])
        for idx in question['sp_predict']:
            p_sp_text.append(document['SENTS'][idx]['text'])
        t_sp.append(t_sp_text)
        p_sp.append(p_sp_text)

In [37]:
all_items[20]['SENTS']

{'QID': 'D037Q03',
 'SENTS': [{'text': '称直系血亲者，', 'start': 0, 'end': 7},
  {'text': '谓己身所从出或从己身所出之血亲。', 'start': 7, 'end': 23},
  {'text': '包括父亲、母亲、儿子、女儿、祖父、祖母、外祖父、外祖母、孙子、孙女、外孙、外孙女。',
   'start': 23,
   'end': 64},
  {'text': '\n称旁系血亲者，', 'start': 64, 'end': 72},
  {'text': '谓非直系血亲，', 'start': 72, 'end': 79},
  {'text': '而与己身出于同源之血亲。', 'start': 79, 'end': 91},
  {'text': '包括同源于父母的兄弟姊妹（含同父异母、同母异父的兄弟姊妹）以及不同辈的叔父、伯父、舅父、姑母、姨母、侄子、侄女、外甥、外甥女等等。',
   'start': 91,
   'end': 156},
  {'text': '\n旁系血亲中的同辈如兄弟姊妹 - 也作兄弟姐妹，', 'start': 156, 'end': 180},
  {'text': '指有相同父亲和母亲的人。', 'start': 180, 'end': 192},
  {'text': '包括双胞胎兄弟姐妹、同父异母兄弟姐妹以及同母异父兄弟姐妹。', 'start': 192, 'end': 221},
  {'text': '\n堂表之分，', 'start': 221, 'end': 227},
  {'text': '同宗是堂，异宗是表。', 'start': 227, 'end': 237},
  {'text': '即父系的男丁是堂，', 'start': 237, 'end': 246},
  {'text': '其他是表。要注意是表姑是父系的表姐妹，', 'start': 246, 'end': 265},
  {'text': '而不是父系的姐妹。', 'start': 265, 'end': 274},
  {'text': '\n堂兄弟姊妹 - 父系血亲的长辈（伯、姆、叔、婶）的儿子或女儿，', 'start': 274, 'end': 

In [29]:
questions[20]

'爸爸的兄弟姊妹的女儿，亲属关系的称谓是什么？'

In [22]:
import pandas as pd

pred_df = pd.DataFrame(index=qids)
pred_df['question'] = questions
pred_df['t_sp'] = t_sp
pred_df['p_sp'] = p_sp

In [24]:
pred_df.head(5)

,question,t_sp,p_sp
D002Q01,苏东坡的老家在哪?,[苏轼回蜀守丧，],[苏轼才20岁，]
D002Q02,苏东坡出生于哪一年?,"[嘉佑二年（1057年），, 苏轼才20岁，]",[苏轼才20岁，]
D002Q03,苏东坡和谁一起进京参加会考?,"[苏轼才20岁，, 与弟弟苏辙一同进京参加会考，]",[与弟弟苏辙一同进京参加会考，]
D002Q04,苏东坡在会考中所写的作文题目为何?,[苏轼以一篇《刑赏忠厚之至论》的论文得到考官梅尧臣的青睐，],[与弟弟苏辙一同进京参加会考，]
D002Q05,苏东坡与曾巩是否同为欧阳修的学生?,"[欧阳修亦十分赞赏，, 原本欲拔擢为第一，, 但又怕该文为自己的门生曾巩所作，, 为了避嫌，...",[当时主试官是欧阳修，]


In [27]:
pred_df.to_csv('syn_pred.csv')

In [16]:
all_sp_predictions[:10]

[[1], [1], [2], [2], [4], [34], [32], [34], [23, 28, 32, 33], [19]]

In [9]:
# all_sp_predictions = []
# all_atype_predictions = []
# for d in tqdm(data):
#     item = extractor.get_item(d)
#     items= [i for i in item]
#     predictions, atypes = extractor.predict(items)
#     all_sp_predictions += predictions
#     all_atype_predictions += atypes

In [10]:
metrics = eval_sp_fgc(all_items, all_sp_predictions)
# atype_accuracy = eval_fgc_atype(all_items, all_atype_predictions)

{'sp_em': 0.20673076923076922, 'sp_prec': 0.5419871794871794, 'sp_recall': 0.5159455128205129, 'sp_f1': 0.49974227161727164}


In [25]:
atype_accuracy

0.8798076923076923

In [ ]:
for d in tqdm(data):
    item = extractor.get_item(d)
    items= [i for i in item]
    for q_id, item in enumerate(items):
        score_list = extractor.predict_score(item)
        d['QUESTIONS'][q_id]['s_score'] = score_list

In [12]:
json_write(data, str(config.RESULT_PATH/"prediction"/"FGC_3_weeks_sup.json"))

In [6]:
data = json_load(config.DATA_ROOT/"FGC"/"FGC_3_weeks.json")

In [11]:
data[0]

{'DID': 'D001',
 'DTEXT': '蘇軾（1037年1月8日－1101年8月24日），眉州眉山（今四川省眉山市）人，北宋時著名的文學家、政治家、藝術家、醫學家。字子瞻，一字和仲，號東坡居士、鐵冠道人。嘉佑二年進士，累官至端明殿學士兼翰林學士，禮部尚書。南宋理學方熾時，加賜諡號文忠，複追贈太師。有《東坡先生大全集》及《東坡樂府》詞集傳世，宋人王宗稷收其作品，編有《蘇文忠公全集》。\n其散文、詩、詞、賦均有成就，且善書法和繪畫，是文學藝術史上的通才，也是公認韻文散文造詣皆比較傑出的大家。蘇軾的散文為唐宋四家（韓愈、柳宗元、歐蘇）之末，與唐代的古文運動發起者韓愈並稱為「韓潮蘇海」，也與歐陽修並稱「歐蘇」；更與父親蘇洵、弟蘇轍合稱「三蘇」，父子三人，同列唐宋八大家。蘇軾之詩與黃庭堅並稱「蘇黃」，又與陸游並稱「蘇陸」；其詞「以詩入詞」，首開詞壇「豪放」一派，振作了晚唐、五代以來綺靡的西崑體餘風。後世與南宋辛棄疾並稱「蘇辛」，惟蘇軾故作豪放，其實清朗；其賦亦頗有名氣，最知名者為貶謫期間借題發揮寫的前後《赤壁賦》。宋代每逢科考常出現其文命題之考試，故當時學者曰：「蘇文熟，喫羊肉、蘇文生，嚼菜羹」。藝術方面，書法名列「蘇、黃、米、蔡」北宋四大書法家（宋四家）之首；其畫則開創了湖州畫派；並在題畫文學史上佔有舉足輕重的地位。',
 'QUESTIONS': [{'QID': 'D001Q09',
   'QTYPE': '基礎題',
   'QTEXT': '辛棄疾是哪一個朝代的人？',
   'ANSWER': [{'ATEXT': '南宋', 'ATEXT_CN': '南宋'}],
   'QTEXT_CN': '辛弃疾是哪一个朝代的人？',
   'QTEXT_RE': '辛弃疾是哪一个朝代的人？',
   'REFINED': 0.9,
   'AMODE': 'Single-Span-Extraction',
   'ATYPE': 'Date-Duration',
   's_score': [0.18831099569797516,
    0.00017647468484938145,
    0.0009485816117376089,
    0.00013792241225019097,
    8.458553929813206e-05,
  

# DRCD

In [7]:
files = os.listdir(config.DATA_ROOT/"release_1.7"/"DRCD")
files = [f for f in files if f.endswith(".json")]

for f in files:
    data = json_load(config.DATA_ROOT/"release_1.7"/"DRCD"/f)
    for d in tqdm(data):
        item = extractor.get_item(d)
        items= [i for i in item]
        for q_id, item in enumerate(items):
            score_list = extractor.predict_score(item)
            d['QUESTIONS'][q_id]['s_score'] = score_list
    json_write(data, str(config.RESULT_PATH/"prediction"/(f[:-5]+"_sup.json")))

 42%|████▏     | 524/1254 [00:27<01:13,  9.95it/s]

tokenized all > 511 id:3_445-1


 85%|████████▍ | 1064/1254 [01:04<00:17, 10.94it/s]

tokenized all > 511 id:7_21053-1


 61%|██████▏   | 491/800 [00:08<00:05, 53.53it/s]

...

 54%|█████▎    | 428/800 [00:07<00:06, 55.73it/s]

tokenized all > 511 id:2_2358-1


 88%|████████▊ | 704/800 [00:12<00:01, 53.83it/s]

tokenized all > 511 id:5_1476-1
tokenized all > 511 id:5_1477-1


100%|██████████| 711/711 [01:22<00:00,  8.60it/s]
